<h1 style="color:blue">Chapter 8. Data Wrangling: Join, Combine, and Reshape</h1>

In [1]:
import pandas as pd
import numpy as np

<h2 style="color:orange">8.1 Hierarchical Indexing</h2>

In [2]:
data = pd.Series(np.random.uniform(size=9),
                 index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])

In [3]:
data

a  1    0.393292
   2    0.969696
   3    0.013896
b  1    0.663792
   3    0.079093
c  1    0.855569
   2    0.836864
d  2    0.212681
   3    0.549301
dtype: float64

In [4]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [5]:
data["b"]

1    0.663792
3    0.079093
dtype: float64

In [6]:
data["b":"c"]

b  1    0.663792
   3    0.079093
c  1    0.855569
   2    0.836864
dtype: float64

In [7]:
data.loc[["b", "d"]]

b  1    0.663792
   3    0.079093
d  2    0.212681
   3    0.549301
dtype: float64

In [8]:
data.loc[:, 2]

a    0.969696
c    0.836864
d    0.212681
dtype: float64

In [9]:
data.unstack()

,1,2,3
a,0.393292,0.969696,0.013896
b,0.663792,NaN,0.079093
c,0.855569,0.836864,NaN
d,NaN,0.212681,0.549301


In [10]:
data.unstack().stack()

a  1    0.393292
   2    0.969696
   3    0.013896
b  1    0.663792
   3    0.079093
c  1    0.855569
   2    0.836864
d  2    0.212681
   3    0.549301
dtype: float64

In [11]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                     index=[["a", "a", "b", "b"],
                            [1, 2, 1, 2]],
                     columns=[["Ohio", "Ohio", "Colorado"],
                              ["Green", "Red", "Green"]])

In [12]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [13]:
frame.index.names = ["key1", "key2"]

In [14]:
frame.columns.names = ["state",  "color"]

In [15]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [16]:
frame.index.nlevels

2

In [17]:
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [18]:
pd.MultiIndex.from_arrays([["Ohio", "Ohio", "Colorade"],
                           ["Green", "Red", "Green"]],
                          names=["state", "color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorade', 'Green')],
           names=['state', 'color'])

<h3 style="color:purple">Reordering and Sorting Levels</h3>

In [19]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [20]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [21]:
frame.swaplevel(0, 1).sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [22]:
frame.sort_index()

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

<h3 style="color:purple">Summary Statistics by Level</h3>

In [23]:
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [24]:
frame.groupby(level="color", axis="columns").sum()

/tmp/ipykernel_5042/775557097.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  frame.groupby(level="color", axis="columns").sum()


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

<h3 style="color:purple">Indexing with a DataFrame's Columns</h3>

In [25]:
frame = pd.DataFrame({"a": range(7), 
                      "b": range(7, 0 , -1), 
                      "c": ["one", "one", "one", "two", "two", "two", "two"],
                      "d": [0, 1, 2, 0, 1, 2, 3]})

In [26]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [27]:
frame2 = frame.set_index(["c", "d"])

In [28]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [29]:
frame.set_index(["c", "d"], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [30]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


In [31]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

<h2 style="color:orange">8.2 Combining and Merging Datasets</h2>

<h3 style="color:purple">Database-Style DataFrame Joins</h3>

In [32]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})

In [33]:
df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})

In [34]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [35]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [36]:
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [37]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})

In [38]:
df4 = pd.DataFrame({"rkey": ["a", "b", "d"], 
                    "data2": pd.Series(range(3), dtype="Int64")})

In [39]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,a,4,a,0
4,a,5,a,0
5,b,6,b,1


In [40]:
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,a,2,0
1,a,4,0
2,a,5,0
3,b,0,1
4,b,1,1
5,b,6,1
6,c,3,<NA>
7,d,<NA>,2


In [41]:
pd.merge(df3, df4, left_on="lkey",  right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,a,2,a,0
1,a,4,a,0
2,a,5,a,0
3,b,0,b,1
4,b,1,b,1
5,b,6,b,1
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


In [42]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                    "data1": pd.Series(range(6), dtype="Int64")})

In [43]:
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"],
                    "data2": pd.Series(range(5), dtype="Int64")})

In [44]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [45]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [46]:
pd.merge(df1, df2, on="key", how="left")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


In [47]:
pd.merge(df1, df2, how="inner")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,a,4,0
7,a,4,2
8,b,5,1
9,b,5,3


In [48]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                     "key2": ["one", "two", "one"],
                     "lval": pd.Series([1, 2, 3], dtype="Int64")})

In [49]:
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                      "key2": ["one", "one","one", "two"],
                      "rval": pd.Series([4, 5, 6, 7], dtype="Int64")})

In [50]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [51]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [52]:
pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,bar,one,3,6
1,bar,two,<NA>,7
2,foo,one,1,4
3,foo,one,1,5
4,foo,two,2,<NA>


In [53]:
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [54]:
pd.merge(left, right, on="key1", suffixes=("_left", "_right"), indicator=True,)

,key1,key2_left,lval,key2_right,rval,_merge
0,foo,one,1,one,4,both
1,foo,one,1,one,5,both
2,foo,two,2,one,4,both
3,foo,two,2,one,5,both
4,bar,one,3,one,6,both
5,bar,one,3,two,7,both


<h3 style="color:purple">Merging on Index
</h3>

In [55]:
left1 = pd.DataFrame({"key": ["a","b","a","a","b","c"], "value":pd.Series(range(6), dtype="Int64")})

In [56]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [57]:
right1 = pd.DataFrame({"group_val":[3.5, 7]}, index=["a", "b"])

In [58]:
right1

,group_val
a,3.5
b,7.0


In [59]:
pd.merge(left1, right1,  left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


In [60]:
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [61]:
lefth = pd.DataFrame(
    {"key1":["Ohio", "Ohio", "Ohio", "Nevada", "Nevada"], 
     "key2":[2000, 2001, 2002, 2001, 2002],
     "data":pd.Series(range(5), dtype="Int64")})

In [62]:
righth_index = pd.MultiIndex.from_arrays(
    [
        ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ]
)

In [63]:
righth = pd.DataFrame({"event1": pd.Series([0, 2, 4, 6, 8, 10],
                                          dtype="Int64", 
                                          index= righth_index),
                       "event2": pd.Series([1, 3, 5, 7, 9, 11], 
                                           dtype="Int64",
                                           index=righth_index)
                      }
                     )
                                           

In [64]:
lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [65]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [66]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [67]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True, how="outer").sort_index()

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1
4,Nevada,2000,<NA>,2,3
4,Nevada,2002,4,<NA>,<NA>


In [68]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6,]],
                     index=["a", "c", "e"],
                     columns=["Ohio", "Nevada"]).astype("Int64")
                        

In [69]:
right2= pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13., 14]],
                     index=["b", "c", "d", "e"],
                     columns=["Missouri", "Alabama"]).astype("Int64")

In [70]:
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [71]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [72]:
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [73]:
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [74]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [75]:
another = pd.DataFrame([[7., 8.], [.9, 10.], [11., 12.], [16, 17.]],
                       index=["a", "c", "e", "f"],
                       columns=["New York", "Oregon"]
                      )

In [76]:
another

,New York,Oregon
a,7.0,8.0
c,0.9,10.0
e,11.0,12.0
f,16.0,17.0


In [77]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,0.9,10.0
e,5,6,13,14,11.0,12.0


In [78]:
left2.join([right2, another], how="outer", sort=True)

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
b,<NA>,<NA>,7,8,NaN,NaN
c,3,4,9,10,0.9,10.0
d,<NA>,<NA>,11,12,NaN,NaN
e,5,6,13,14,11.0,12.0
f,<NA>,<NA>,<NA>,<NA>,16.0,17.0


In [79]:
arr = np.arange(12).reshape((3, 4))

In [80]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [81]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [82]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")

In [83]:
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")

In [84]:
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")

In [85]:
s1

a    0
b    1
dtype: Int64

In [86]:
s2

c    2
d    3
e    4
dtype: Int64

In [87]:
s3

f    5
g    6
dtype: Int64

In [88]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: Int64

In [89]:
pd.concat([s1, s2, s3], axis="columns")

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [90]:
s4 = pd.concat([s1, s3])

In [91]:
s4

a    0
b    1
f    5
g    6
dtype: Int64

In [92]:
pd.concat([s1, s4], axis="columns")

,0,1
a,0,0
b,1,1
f,<NA>,5
g,<NA>,6


In [93]:
pd.concat([s1, s4], axis="columns", join="inner")

,0,1
a,0,0
b,1,1


In [94]:
result = pd.concat([s1, s1, s3], keys=["one", "two", "three"])

In [95]:
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: Int64

In [96]:
result.unstack()

,a,b,f,g
one,0,1,<NA>,<NA>
two,0,1,<NA>,<NA>
three,<NA>,<NA>,5,6


In [97]:
pd.concat([s1, s2, s3], axis="columns", keys=["one", "two", "three"])

,one,two,three
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [98]:
df1 = pd.DataFrame(np.arange(6).reshape(3,2), index=["a", "b", "c"], columns=["one", "two"])

In [99]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2),
                   index=["a", "c"],
                   columns=["three", "four"]
)

In [100]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [101]:
df2

,three,four
a,5,6
c,7,8


In [102]:
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [103]:
pd.concat({"level 1":df1, "level 2":df2}, axis="columns")

level 1     level 2     
      one two   three four
a       0   1     5.0  6.0
b       2   3     NaN  NaN
c       4   5     7.0  8.0

In [104]:
pd.concat([df1, df2], axis="columns", keys=["level 1", "level2"], names=["Upper", "Lower"])

Upper level 1     level2     
Lower     one two  three four
a           0   1    5.0  6.0
b           2   3    NaN  NaN
c           4   5    7.0  8.0

In [105]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [106]:
df2

,three,four
a,5,6
c,7,8


In [107]:
pd.concat([df1, df2], axis=1, keys=["Level 1", "Level 2"], names=["Upper", "Lower"])

Upper Level 1     Level 2     
Lower     one two   three four
a           0   1     5.0  6.0
b           2   3     NaN  NaN
c           4   5     7.0  8.0

In [108]:
df1 = pd.DataFrame(np.random.standard_normal((3,4)), columns=["a", "b", "c", "d"])

In [109]:
df2 = pd.DataFrame(np.random.standard_normal((2,3)), columns=["b", "d", "a"])

In [110]:
df1

,a,b,c,d
0,1.251539,0.127574,1.806894,-0.006547
1,0.332987,1.159568,-0.221706,1.970860
2,-2.480512,0.875602,-1.990252,1.060724


In [111]:
df2

,b,d,a
0,-0.439400,0.589475,0.605822
1,0.018819,-1.667031,0.281390


In [112]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,1.251539,0.127574,1.806894,-0.006547
1,0.332987,1.159568,-0.221706,1.970860
2,-2.480512,0.875602,-1.990252,1.060724
3,0.605822,-0.439400,NaN,0.589475
4,0.281390,0.018819,NaN,-1.667031


<h3 style="color:purple">Combining Data with Ovelap
</h3>

In [113]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan], index=["a", "b", "c", "d", "e", "f"])

In [114]:
b = pd.Series([0, np.nan, 2., np.nan, np.nan, 5.,], index=["a", "b", "c", "d", "e", "f"]) 

In [115]:
a

a    NaN
b    2.5
c    0.0
d    3.5
e    4.5
f    NaN
dtype: float64

In [116]:
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [117]:
np.where(pd.isna(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

In [118]:
a.combine_first(b)

a    0.0
b    2.5
c    0.0
d    3.5
e    4.5
f    5.0
dtype: float64

In [119]:
df1 = pd.DataFrame({"a":[1., np.nan, 5., np.nan],
                    "b":[np.nan, 2., np.nan, 6.],
                    "c":range(2, 18, 4)})

In [120]:
df2 = pd.DataFrame({"a":[5., 4., np.nan, 3., 7.], 
                    "b":[np.nan, 3., 4., 6., 8.]})

In [121]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [122]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [123]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


<h2 style="color:orange">8.3 Reshaping and Pivoting</h2>

<h3 style="color:purple">Reshaping with Hierarchical Indexing</h3>

In [124]:
data = pd.DataFrame(np.arange(6).reshape((2,3)),
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"],
                                     name="number"))

In [125]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [126]:
result=data.stack()

In [127]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [128]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [129]:
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [130]:
result.unstack(level="state")

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [131]:
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d",], dtype="Int64")
s2 = pd.Series([4, 5, 6], index= ["c", "d", "e"], dtype="Int64") 

In [132]:
data2 = pd.concat([s1, s2], keys=["one", "two"])

In [133]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [134]:
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,<NA>
two,<NA>,<NA>,4,5,6


In [135]:
data2.unstack().stack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [136]:
data2.unstack().stack(dropna=False)

/tmp/ipykernel_5042/3936770077.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data2.unstack().stack(dropna=False)


one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64

In [137]:
df=pd.DataFrame({"left":result, "right":result + 5},
                columns=pd.Index(["left", "right"], name="side"))

In [138]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [139]:
df.unstack(level="state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [140]:
df.unstack(level="state").stack(level="side",future_stack="True")

state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

In [141]:
data = pd.read_csv("../examples/macrodata.csv")

In [142]:
data = data.loc[:,["year", "quarter", "realgdp", "infl", "unemp"]]

In [143]:
data.head()

,year,quarter,realgdp,infl,unemp
0,1959,1,2710.349,0.00,5.8
1,1959,2,2778.801,2.34,5.1
2,1959,3,2775.488,2.74,5.3
3,1959,4,2785.204,0.27,5.6
4,1960,1,2847.699,2.31,5.2


In [144]:
periods = pd.PeriodIndex(year=data.pop("year"), 
                         quarter=data.pop("quarter"),
                         name="date")                         

/tmp/ipykernel_5042/1091193715.py:1: FutureWarning: Constructing PeriodIndex from fields is deprecated. Use PeriodIndex.from_fields instead.
  periods = pd.PeriodIndex(year=data.pop("year"),


In [145]:
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [146]:
data.index = periods.to_timestamp("D")

In [147]:
data.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [148]:
data = data.reindex(columns=["realgdp", "infl", "unemp"])

In [149]:
data.columns.name = "item"

In [150]:
data.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [151]:
long_data = (data.stack()
             .reset_index()
             .rename(columns={0: "value"}))

In [152]:
long_data[:10]

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
5,1959-04-01,unemp,5.100
6,1959-07-01,realgdp,2775.488
7,1959-07-01,infl,2.740
8,1959-07-01,unemp,5.300
9,1959-10-01,realgdp,2785.204


In [153]:
pivoted = long_data.pivot(index="date", columns="item", values="value")

In [154]:
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


In [155]:
long_data["value_2"] =np.random.standard_normal(len(long_data))

In [156]:
long_data[:10]

,date,item,value,value_2
0,1959-01-01,realgdp,2710.349,1.117492
1,1959-01-01,infl,0.000,0.041860
2,1959-01-01,unemp,5.800,0.090915
3,1959-04-01,realgdp,2778.801,-0.326696
4,1959-04-01,infl,2.340,0.141815
5,1959-04-01,unemp,5.100,-0.193721
6,1959-07-01,realgdp,2775.488,0.125651
7,1959-07-01,infl,2.740,0.531682
8,1959-07-01,unemp,5.300,0.066012
9,1959-10-01,realgdp,2785.204,-1.653314


In [157]:
pivoted = long_data.pivot(index="date", columns="item")

In [158]:
pivoted.head()

value                   value_2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  0.041860  1.117492  0.090915
1959-04-01  2.34  2778.801   5.1  0.141815 -0.326696 -0.193721
1959-07-01  2.74  2775.488   5.3  0.531682  0.125651  0.066012
1959-10-01  0.27  2785.204   5.6  0.388068 -1.653314  0.184182
1960-01-01  2.31  2847.699   5.2 -1.489540 -2.146547 -0.023884

In [161]:
pivoted["value"].head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


In [162]:
unstacked = long_data.set_index(["date", "item"]).unstack(level="item") 

In [165]:
unstacked.head()

value                   value_2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  0.041860  1.117492  0.090915
1959-04-01  2.34  2778.801   5.1  0.141815 -0.326696 -0.193721
1959-07-01  2.74  2775.488   5.3  0.531682  0.125651  0.066012
1959-10-01  0.27  2785.204   5.6  0.388068 -1.653314  0.184182
1960-01-01  2.31  2847.699   5.2 -1.489540 -2.146547 -0.023884

<h3 style="color:purple">Pivoting "Wide" to "Long" Format</h3>

In [169]:
df = pd.DataFrame({"key":["foo", "bar", "baz"],
                    "A":[1, 2, 3],
                    "B":[4, 5, 6],
                    "C":[7, 8, 9]})

In [170]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [171]:
melted = pd.melt(df, id_vars="key")

In [173]:
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9
